In [33]:
import os 
import re
import pandas as pd

import matplotlib.pyplot as plt

# Get data

In [2]:
files = os.listdir('./data')

In [140]:
df_raw = pd.DataFrame()

for file in files:
    new_df = pd.read_excel('./data/' + file)
    df_raw = pd.concat([df_raw, new_df])

# Treat data

In [240]:
df = df_raw.dropna(subset=["Country Code"])

years_columns = [c for c in df.columns if re.search(r"\d+", c)]
columns = ["Country Code", "Series Code", *years_columns]

df = df.loc[:, columns].replace("..", 'NaN')

# treat years
years = [int(y.split()[0]) for y in years_columns]
years.sort()

rename_dict = {yn: y for yn, y in zip(years_columns, years)}
df = df.rename(rename_dict, axis=1)

df = df.dropna(subset=years, how="all")

df = df.drop_duplicates(['Country Code', 'Series Code'], keep='last')

# df = df.reindex(sorted(df.columns), axis=1).set_index(["Country Code", "Series Code"])

In [241]:
# select vars
vars = [
    "BAR.NOED.15UP.ZS",  # Barro-Lee: Percentage of population age 15+ with no education
    "BAR.NOED.75UP.ZS",  # Barro-Lee: Percentage of population age 75+ with no education
    "NY.ADJ.ICTR.GN.ZS",  # Adjusted savings: gross savings (% of GNI)
    "NY.GDP.MKTP.KD.ZG",  # GDP growth (annual %)
    "NY.GDP.PCAP.KD",  # GDP per capita (constant 2005 US$)
    "NY.ADJ.NNTY.KD.ZG",  # Adjusted net national income (annual % growth)
    "NY.ADJ.NNTY.PC.KD",  # Adjusted net national income per capita (constant 2015 US$)
    "SE.TER.ENRR",  # Gross enrolment ratio for tertiary education, both sexes (%)
    "NE.GDI.TOTL.ZS",  # Gross capital formation (% of GDP
]

df = df[df["Series Code"].isin(vars)].set_index(['Country Code', 'Series Code']).astype('float')

df = df.iloc[:, :63]

# Interpolation

In [243]:
df_interpolated = df.infer_objects(copy=False).interpolate(axis=1,limit_direction='both')

In [ ]:
var = 'BAR.NOED.15UP.ZS'

for var in vars:

    print('\n\nVAR:', var)
    df_plot = df.filter(regex=var,axis=0)

    df_plot.T.plot()

    df_interpolated.filter(regex=var,axis=0).T.plot()

    plt.show()

# Melt both dataframes

In [245]:
df_long = df.reset_index().melt(id_vars=['Country Code', 'Series Code'], value_vars=df.columns, var_name='Year')
df_interpolated_long = df_interpolated.reset_index().melt(id_vars=['Country Code', 'Series Code'], value_vars=df_interpolated.columns, var_name='Year')